In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim
import csv
from datetime import datetime

def setup_directories():
    os.makedirs('results/equalized', exist_ok=True)
    os.makedirs('results/plots', exist_ok=True)
    os.makedirs('results/metrics', exist_ok=True)

def load_image_pairs(low_dir='/content/drive/MyDrive/data/low', high_dir='/content/drive/MyDrive/data/high'):
    low_images = sorted([f for f in os.listdir(low_dir) if f.endswith(('.png', '.jpg', '.jpeg'))])
    high_images = sorted([f for f in os.listdir(high_dir) if f.endswith(('.png', '.jpg', '.jpeg'))])

    pairs = []
    for low_img, high_img in zip(low_images, high_images):
        low_path = os.path.join(low_dir, low_img)
        high_path = os.path.join(high_dir, high_img)
        pairs.append((low_path, high_path))

    return pairs


In [11]:
def enhance_image(image):
    enhanced = cv2.equalizeHist(image)
    return enhanced

def save_comparison_plot(original, enhanced, ground_truth, ssim_score, filename):
    plt.figure(figsize=(15, 5))

    plt.subplot(1, 3, 1)
    plt.imshow(original, cmap='gray')
    plt.title('Original Image')
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.imshow(enhanced, cmap='gray')
    plt.title(f'Enhanced\nSSIM: {ssim_score:.4f}')
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.imshow(ground_truth, cmap='gray')
    plt.title('Ground Truth')
    plt.axis('off')

    plot_path = os.path.join('results', 'plots', f'{filename}_comparison.png')
    plt.savefig(plot_path, bbox_inches='tight', dpi=150)
    plt.close()

def log_metrics(filename, ssim_score, log_file='results/metrics/metrics.csv'):
    file_exists = os.path.isfile(log_file)

    with open(log_file, 'a', newline='') as csvfile:
        fieldnames = ['timestamp', 'image_name', 'ssim']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        if not file_exists:
            writer.writeheader()

        writer.writerow({
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'image_name': filename,
            'ssim': ssim_score
        })

In [12]:
def process_images():
    setup_directories()
    image_pairs = load_image_pairs()

    print(f"Found {len(image_pairs)} image pairs to process")

    for low_path, high_path in image_pairs:
        original = cv2.imread(low_path, cv2.IMREAD_GRAYSCALE)
        ground_truth = cv2.imread(high_path, cv2.IMREAD_GRAYSCALE)

        if original is None or ground_truth is None:
            print(f"Warning: Could not load image pair {low_path} and {high_path}")
            continue

        enhanced = enhance_image(original)
        ssim_score = ssim(enhanced, ground_truth, data_range=255)
        filename = os.path.splitext(os.path.basename(low_path))[0]

        enhanced_path = os.path.join('results', 'equalized', f'{filename}_equalized.png')
        cv2.imwrite(enhanced_path, enhanced)

        save_comparison_plot(original, enhanced, ground_truth, ssim_score, filename)

        log_metrics(filename, ssim_score)

        print(f"Processed {filename}: SSIM = {ssim_score:.4f}")

In [13]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
process_images()


Found 15 image pairs to process
Processed 102: SSIM = 0.4411
Processed 13: SSIM = 0.4025
Processed 18: SSIM = 0.6573
Processed 2: SSIM = 0.5927
Processed 21: SSIM = 0.5915
Processed 39: SSIM = 0.5474
Processed 43: SSIM = 0.6929
Processed 48: SSIM = 0.5581
Processed 5: SSIM = 0.6478
Processed 52: SSIM = 0.7656
Processed 54: SSIM = 0.7464
Processed 57: SSIM = 0.6309
Processed 60: SSIM = 0.5884
Processed 75: SSIM = 0.5603
Processed 83: SSIM = 0.5240
